### Universidad Nacional de Córdoba - Facultad de Matemática, Astronomía, Física y Computación

#### Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones 2020

Búsqueda y Recomendación de Textos Legales - Análisis y Curación de Datos

Mentor: Claudio Sarate

Integrantes:
* Clara Quintana
* Ezequiel Juarez
* David Veisaga
* Jorge Pérez 

### Práctico de Introducción al Aprendizaje Automático

El objetivo de este práctico es desarrollar distintos modelos de clasificación para poder evaluar la performance y la exactitud de predicción de cada modelo.

### Requisitos iniciales

El corpus debe estar ya depurado y debe poseer una columna con clases definidas previamente.

Nota: es importante que la cantidad de las distintas clases sea medianamente balanceada para que el entrenamiento sea lo mas eficiente posible.

In [1]:
import sys
import pandas

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn import ensemble
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
# Se verfica entorno de ejecución
in_colab = "google.colab" in sys.modules

if in_colab:
    from google.colab import drive

    drive.mount("/content/drive")
    BASE_DIR = "/content/drive/My Drive/Diplo2020 Mentoria/"
else:
    BASE_DIR = "../"

In [3]:
train_data = BASE_DIR + "corpus3.csv"
dataset = pandas.read_csv(train_data)
dataset.head()

,Unnamed: 0,TEXTO,DOCUMENTO,TIPO
0,0,sala electoral comp.originaria tribunal superi...,../Datos/Electoral//A 001-2018 COOPI C MUN CAR...,AUTO
1,1,sala electoral comp.originaria tribunal superi...,../Datos/Electoral//A 002-2018 Denuncia R R RA...,AUTO
2,2,sala electoral comp.originaria tribunal superi...,../Datos/Electoral//A 003-2018 VEDIA FLORES Cr...,AUTO
3,3,sala electoral comp.originaria tribunal superi...,../Datos/Electoral//A 004-2018 FERNANDEZ Favio...,AUTO
4,4,sala electoral comp.originaria tribunal superi...,../Datos/Electoral//A 005-2018 ATANOR ADI ref.pdf,AUTO


In [4]:
dataset['TIPO'].value_counts()

AUTO         125
SENTENCIA     25
Name: TIPO, dtype: int64

### Enunciado del práctico

 ----------------------------------------------------------------------------------------------------------

Transformar el texto en vectores numéricos utilizando scikit-learn. Los procesos de vectorización, clasificación y evaluación de performance pueden ser hechos paso a paso o mediante el uso de pipelines para mayor eficiencia.

Scikit-learn ofrece 3 modelos de vectorización:

* *CountVectorizer*: Convert a collection of text documents to a matrix of token counts
* *TfidfVectorizer*: Convert a collection of raw documents to a matrix of TF-IDF features.
*  *HashingVectorizer*: Convert a collection of text documents to a matrix of token occurrences

Comparamos los 3 modelos usando el primer documento del corpus.

In [ ]:
# Texto del primer documento
texto = dataset[0:1].TEXTO

*CountVectorizer*

El recuento de palabras es un buen punto de partida, pero es muy básico. Un problema con los recuentos simples es que algunas palabras como “testamento” aparecerán muchas veces y sus recuentos grandes no serán muy significativos en los vectores codificados.

In [ ]:
vectorizer_1 = CountVectorizer()

In [ ]:
# tokenizar y construir el vocabulario
vectorizer_1.fit(texto)

In [ ]:
# resumen
print(vectorizer_1.vocabulary_)

In [ ]:
# codificador de documentos
vector_1 = vectorizer_1.transform(texto)

In [ ]:
# resumir vector codificado
print(vector_1.shape)
print(type(vector_1))
print(vector_1.toarray())

*TfidfVectorizer*

TF-IDF es un acrónimo que significa Frecuencia de Término – Frecuencia Inversa de Documento que son los componentes de las puntuaciones resultantes asignadas a cada palabra.

* **Término Frecuencia**: Esto resume la frecuencia con la que una palabra dada aparece dentro de un documento.
* **Frecuencia inversa de documentos**: Esto reduce la escala de las palabras que aparecen mucho en los documentos.

El peso que tiene cada palabra ($w_i,_j$) es directamente proporcional a las veces que aparece en el documento ($tf_i,_j$) e inversamente proporcional a las veces que aparece en todos los documentos ($df_i$).

$$ w_i,_j = tf_i,_j * log \frac{N}{df_i} $$

$tf_i,_j$ Frecuencia de la palabra $i$ en el documento $j$.

$df_i$ Número de documentos que contienen la palabra $i$.

$N$ Número total de documentos.


TF-IDF son puntuaciones de frecuencia de palabras que tratan de resaltar las palabras que son más interesantes, por ejemplo, frecuentes en un documento pero no en todos los documentos.

Los recuentos y las frecuencias pueden ser muy útiles, pero una limitación de estos métodos es que el vocabulario puede llegar a ser muy amplio. Esto, a su vez, requerirá grandes vectores para codificar los documentos e impondrá grandes requisitos a la memoria y a los algoritmos de ralentización. 

In [ ]:
vectorizer_2 = TfidfVectorizer()

In [ ]:
# tokenizar y construir el vocabulario
vectorizer_2.fit(texto)

In [ ]:
# resumir
print(vectorizer_2.vocabulary_)
print(vectorizer_2.idf_)

In [ ]:
# documento codificado
vector_2 = vectorizer_2.transform(texto)

In [ ]:
# resumir vector codificado
print(vector_2.shape)
print(vector_2.toarray())

*HashingVectorizer*

Podemos usar un hash de palabras para convertirlas en números enteros. La ventaja de esto, es que permite no tener vocabulario y poder elegir un vector de longitud fija arbitraria. Una desventaja es que el hash es una función unidireccional, por lo que no hay forma de volver a convertir la codificación en una palabra.

La clase HashingVectorizer implementa este enfoque que se puede utilizar para convertir palabras en hash de forma coherente y, a continuación, convertir en token y codificar documentos según sea necesario.

In [ ]:
vectorizer_3 = HashingVectorizer(n_features=300)

In [ ]:
# documento codificado
vector_3 = vectorizer_3.transform(texto)

In [ ]:
# resumir vector codificado
print(vector_3.shape)
print(vector_3.toarray())

Al ejecutar el ejemplo se codifica el documento de muestra como una matriz dispersa de 300 elementos. Los valores del documento codificado corresponden a recuentos de palabras normalizados por defecto en el rango de -1 a 1, pero se pueden hacer recuentos enteros simples cambiando la configuración por defecto.

De los 3 métodos posibles utilizaremos TfidfVectorizer

### Dividir los datos en entrenamiento y validación con un procentaje de 70% para entrenamiento y 30% para validación con shuffle, seleccionar las features X e Y. 

In [100]:
vectorizer = TfidfVectorizer()
vectorizer.fit_transform(dataset['TEXTO'])

<150x19220 sparse matrix of type '<class 'numpy.float64'>'
	with 204569 stored elements in Compressed Sparse Row format>

In [101]:
# División entre instancias vectorizadas y etiquetas
X, y = vectorizer.transform(dataset["TEXTO"]), dataset["TIPO"]

In [102]:
# división entre entrenamiento y evaluación
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, shuffle=True)

In [103]:
# Dividimos en 5 subgrupos
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

 ----------------------------------------------------------------------------------------------------------

### Clasificar utilizando los datos de entrenamiento mediante Logistic Regresion, Naive Bayes, Random Forest y SVM.

In [104]:
def calculo_metricas(tn, fp, fn, tp):
    accuracy = 0
    precision = 0
    recall = 0
    f1 = 0

    if (tp + fp + fn + tn) > 0:
        accuracy = (tp + tn) / (tp + fp + fn + tn)
    if (tp + fp) > 0:
        precision = (tp) / (tp + fp) 
    if (tp + fn) > 0:
        recall = (tp) / (tp + fn)
    if (precision + recall) > 0:
        f1 = (2 * precision * recall) / (precision + recall)

    return accuracy, precision, recall, f1

In [105]:
def imprimir_metricas(titulo, y_train, y_train_pred, y_test, y_test_pred):
    print(titulo)
    print('-' * len(titulo))

    print('\nEntrenamiento')
    tn, fp, fn, tp = confusion_matrix(y_train, y_train_pred).ravel()

    print(classification_report(y_train, y_train_pred))
    print(f'Matriz de Confusión \n\tTP:{tp:{3}} \tFP:{fp:{3}} \n\tTN:{tn:{3}} \tFN:{fn:{3}}')

    print('\nEvaluación')
    tn, fp, fn, tp = confusion_matrix(y_test, y_test_pred).ravel()

    print(classification_report(y_test, y_test_pred))
    print(f'Matriz de Confusión \n\tTP:{tp:{3}} \tFP:{fp:{3}} \n\tTN:{tn:{3}} \tFN:{fn:{3}}')    
    
    return tn, fp, fn, tp

Defino una función para cada uno de los modelos a comparar. 

* Los parámetros de entrada son los datos de entrenaimiento y evaluación.

* Los parámetros de salida son las métricas de los datos de evaluación.


In [106]:
def modelo_LogisticRegretion(X_train, y_train, X_test, y_test):
    model = LogisticRegression(solver='lbfgs')
    model.fit(X_train, y_train)
   
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)    

    tn, fp, fn, tp = imprimir_metricas('Modelo Logistic Regretion', y_train, y_train_pred, y_test, y_test_pred)
    accuracy, precision, recall, f1 = calculo_metricas(tn, fp, fn, tp)
    
    return tp, fp, tn, fn, accuracy, precision, recall, f1

In [107]:
def modelo_NaiveBayes(X_train, y_train, X_test, y_test):
    model = MultinomialNB()
    model.fit(X_train, y_train)
    
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)    

    tn, fp, fn, tp = imprimir_metricas('Modelo Naive Bayes', y_train, y_train_pred, y_test, y_test_pred)
    accuracy, precision, recall, f1 = calculo_metricas(tn, fp, fn, tp)
        
    return tp, fp, tn, fn, accuracy, precision, recall, f1

In [108]:
def modelo_RandomForest(X_train, y_train, X_test, y_test):
    model = ensemble.RandomForestClassifier(random_state=0)
    model.fit(X_train, y_train)
    
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)    

    tn, fp, fn, tp = imprimir_metricas('Modelo Random Forest', y_train, y_train_pred, y_test, y_test_pred)
    accuracy, precision, recall, f1 = calculo_metricas(tn, fp, fn, tp)

    return tp, fp, tn, fn, accuracy, precision, recall, f1    

In [109]:
def modelo_SVM(X_train, y_train, X_test, y_test):
    model = svm.SVC()
    model.fit(X_train, y_train)
    
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)    

    tn, fp, fn, tp = imprimir_metricas('Modelo SVM', y_train, y_train_pred, y_test, y_test_pred)
    accuracy, precision, recall, f1 = calculo_metricas(tn, fp, fn, tp)
        
    return tp, fp, tn, fn, accuracy, precision, recall, f1    

 ----------------------------------------------------------------------------------------------------------

### Realizar las predicciones para cada caso generando la matriz de confusión (plotear) y los reportes de performance con valores para precision, recall, f1-score y support.

Guardaremos las metrícas en la siguiente variable:

In [110]:
model_list = []

In [111]:
# Entreno con Logistic Regretion
tp, fp, tn, fn, accuracy, precision, recall, f1 = modelo_LogisticRegretion(X_train, y_train, X_test, y_test)
model_list.append(['Logistic Regretion', tp, fp, tn, fn, accuracy, precision, recall, f1])

Modelo Logistic Regretion
-------------------------

Entrenamiento
              precision    recall  f1-score   support

        AUTO       0.91      0.98      0.94        86
   SENTENCIA       0.85      0.58      0.69        19

   micro avg       0.90      0.90      0.90       105
   macro avg       0.88      0.78      0.82       105
weighted avg       0.90      0.90      0.90       105

Matriz de Confusión 
	TP: 11 	FP:  2 
	TN: 84 	FN:  8

Evaluación
              precision    recall  f1-score   support

        AUTO       0.95      1.00      0.97        39
   SENTENCIA       1.00      0.67      0.80         6

   micro avg       0.96      0.96      0.96        45
   macro avg       0.98      0.83      0.89        45
weighted avg       0.96      0.96      0.95        45

Matriz de Confusión 
	TP:  4 	FP:  0 
	TN: 39 	FN:  2


In [112]:
# Entreno con Naive Bayes
tp, fp, tn, fn, accuracy, precision, recall, f1 = modelo_NaiveBayes(X_train, y_train, X_test, y_test)
model_list.append(['Naive Bayes', tp, fp, tn, fn, accuracy, precision, recall, f1])

Modelo Naive Bayes
------------------

Entrenamiento
              precision    recall  f1-score   support

        AUTO       0.82      1.00      0.90        86
   SENTENCIA       0.00      0.00      0.00        19

   micro avg       0.82      0.82      0.82       105
   macro avg       0.41      0.50      0.45       105
weighted avg       0.67      0.82      0.74       105

Matriz de Confusión 
	TP:  0 	FP:  0 
	TN: 86 	FN: 19

Evaluación
              precision    recall  f1-score   support

        AUTO       0.87      1.00      0.93        39
   SENTENCIA       0.00      0.00      0.00         6

   micro avg       0.87      0.87      0.87        45
   macro avg       0.43      0.50      0.46        45
weighted avg       0.75      0.87      0.80        45

Matriz de Confusión 
	TP:  0 	FP:  0 
	TN: 39 	FN:  6


C:\Users\jperezv\Anaconda3\envs\mentoria\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [113]:
# Entreno con Random Forest
tp, fp, tn, fn, accuracy, precision, recall, f1 = modelo_RandomForest(X_train, y_train, X_test, y_test)
model_list.append(['Random Forest', tp, fp, tn, fn, accuracy, precision, recall, f1])

Modelo Random Forest
--------------------

Entrenamiento
              precision    recall  f1-score   support

        AUTO       0.99      1.00      0.99        86
   SENTENCIA       1.00      0.95      0.97        19

   micro avg       0.99      0.99      0.99       105
   macro avg       0.99      0.97      0.98       105
weighted avg       0.99      0.99      0.99       105

Matriz de Confusión 
	TP: 18 	FP:  0 
	TN: 86 	FN:  1

Evaluación
              precision    recall  f1-score   support

        AUTO       0.95      1.00      0.97        39
   SENTENCIA       1.00      0.67      0.80         6

   micro avg       0.96      0.96      0.96        45
   macro avg       0.98      0.83      0.89        45
weighted avg       0.96      0.96      0.95        45

Matriz de Confusión 
	TP:  4 	FP:  0 
	TN: 39 	FN:  2


C:\Users\jperezv\Anaconda3\envs\mentoria\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [114]:
# Entreno con SVM
tp, fp, tn, fn, accuracy, precision, recall, f1 = modelo_SVM(X_train, y_train, X_test, y_test)
model_list.append(['SVM', tp, fp, tn, fn, accuracy, precision, recall, f1])

Modelo SVM
----------

Entrenamiento
              precision    recall  f1-score   support

        AUTO       0.82      1.00      0.90        86
   SENTENCIA       0.00      0.00      0.00        19

   micro avg       0.82      0.82      0.82       105
   macro avg       0.41      0.50      0.45       105
weighted avg       0.67      0.82      0.74       105

Matriz de Confusión 
	TP:  0 	FP:  0 
	TN: 86 	FN: 19

Evaluación
              precision    recall  f1-score   support

        AUTO       0.87      1.00      0.93        39
   SENTENCIA       0.00      0.00      0.00         6

   micro avg       0.87      0.87      0.87        45
   macro avg       0.43      0.50      0.46        45
weighted avg       0.75      0.87      0.80        45

Matriz de Confusión 
	TP:  0 	FP:  0 
	TN: 39 	FN:  6


C:\Users\jperezv\Anaconda3\envs\mentoria\lib\site-packages\sklearn\daal4sklearn\svm.py:332: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\jperezv\Anaconda3\envs\mentoria\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


 ----------------------------------------------------------------------------------------------------------

### Determinar el modelo con mejor performance.

In [115]:
df = pandas.DataFrame(model_list, columns=['modelo', 'tp', 'fp', 'tn', 'fn', 'accuracy', 'precision', 'recall', 'f1'])
df.sort_values(by='accuracy', ascending=False)

,modelo,tp,fp,tn,fn,accuracy,precision,recall,f1
0,Logistic Regretion,4,0,39,2,0.955556,1.0,0.666667,0.8
2,Random Forest,4,0,39,2,0.955556,1.0,0.666667,0.8
1,Naive Bayes,0,0,39,6,0.866667,0.0,0.000000,0.0
3,SVM,0,0,39,6,0.866667,0.0,0.000000,0.0


 ----------------------------------------------------------------------------------------------------------

### Probar con y sin shuffle en la partición de los datos y con distintos hiperparámetros para ver si los resultados cambian de acuerdo a si los datos se han mezclado al entrenar y validar o no.

 ----------------------------------------------------------------------------------------------------------

## Conclusiones

 ----------------------------------------------------------------------------------------------------------

### Entrega

Formato de entrega: Deberán utilizar esta notebook con los códigos con los que hicieron el análisis y los anaálisis y conclusiones despues de cada proceso. 

Fecha de entrega: 16/8

 ----------------------------------------------------------------------------------------------------------

In [116]:
dataset[0:1].TIPO, dataset[0:1].TEXTO


(0    AUTO
 Name: TIPO, dtype: object,
 0    sala electoral comp.originaria tribunal superi...
 Name: TEXTO, dtype: object)

In [117]:
model = ensemble.RandomForestClassifier(random_state=0)
model.fit(X_train, y_train)

C:\Users\jperezv\Anaconda3\envs\mentoria\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [118]:
model.predict(X_train[0:1])

array(['AUTO'], dtype=object)

In [121]:
vectorizer.transform(dataset[0:1].TEXTO)

<1x767 sparse matrix of type '<class 'numpy.float64'>'
	with 767 stored elements in Compressed Sparse Row format>

In [122]:
model.predict(vectorizer.transform(dataset[0:1].TEXTO))

ValueError: Number of features of the model must match the input. Model n_features is 19220 and input n_features is 767 